In [1]:
# CounterVectorizer

# TfidfVectorizer

# multinomal Naive Bayes : 확률 모델
# LogisticRegression : 다중 클래스 - 회기 기반 분류
# RifgeClassifer : L2규제 - 회기 기반 분류
    # L2규제 ^  가중치의 제곱의 값을 추가로 패널티를 부여
# N-gram : ^
    # 단점 : 차원 폭발에 주의 (정규화 / 차원 축소 고려)

# -----
# Kolnpy Okt : ^
    # 사용하려고 할시 추가 설정 필요 :

In [2]:
# %pip install scikit-learn
# %pip install nltk

### 5. BOW 기반의 문서 분류

#### 1) 라이브러리 설치

In [3]:
import numpy as np
import pandas as pd
# scikit - learn
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split

# BOW 변환 모델
# CountVectorizer
# TfidfTransformer

# 분류 모델
# multinomal Naive Bayes : 확률 모델
# LogisticRegression : 다중 클래스 - 회기 기반 분류
# RifgeClassifer : L2규제 - 회기 기반 분류
    # L2규제 ^  가중치의 제곱의 값을 추가로 패널티를 부여
# N-gram : 문장을 연속된 N개의 단어(또는 문자) 단위로 나누어,
         # 이들의 등장 확률 패턴을 기반으로 문서나 문장을 분류하는 통계적 언어 모델.
    # 단점 : 차원 폭발에 주의 (정규화 / 차원 축소 고려)
# Decisiontree ^
# RandomForest ^
# GradientBoosting ^

from sklearn.naive_bayes import MultinomialNB
from sklearn. linear_model import LogisticRegression,RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

#### 2) 데이터셋 불러오기 : fetch_20newsgroups

In [4]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

# data load
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove = ('headers','footers','quetes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                      remove = ('headers','footers','quetes'),
                                      categories=categories)

```
alt.atheism 무신론, 종교 비판 토론 종교의 존재, 신의 존재 유무, 종교적 주장 반박, 철학적 논쟁 등
talk.religion.misc 일반 종교 토론 (기타 잡담 포함) 기독교, 불교, 이슬람 등 다양한 종교 관련 이야기, 개인 경험, 신념 공유 등
comp.graphics 컴퓨터 그래픽스, 이미지 처리 3D 렌더링, 이미지 파일 포맷, 그래픽 소프트웨어 사용법, OpenGL 등 기술 관련 토론
sci.space 우주 과학, 천문학 로켓, NASA, 행성 탐사, 외계 생명 가능성, 우주 물리학 관련 토론
```

In [5]:
# 파일을 직접 다운로드 받아서 데이터셋을 불러오는 방법
# from sklearn.datasets import load_files
# train_path = r'설치된 파일 경로'
# test_path = r'설치된 파일 경로'
# newsgroups_train = load_files(train_path,encoding='latin1')
# newsgroups_test = load_files(test_path,encoding='latin1')

In [6]:
newsgroups_train.target_names

['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']

In [7]:
# # 카테고리 제거
# def filter_categories(dataset, categories):
#     target_names = dataset.target_names
#     selected_idx = [ target_names.index(c) for c in categories  ]
#     #필터링
#     data_filtered, target_filtered = [], []
#     for text,label in zip(dataset.data, dataset.target):
#         if label in selected_idx:
#             new_label = selected_idx.index(label)  # 라벨 재 정렬
#             data_filtered.append(text) ; target_filtered.append( new_label  )
#     return data_filtered,target_filtered,categories

In [8]:
# 카테고리 제거
def filter_categories(dataset, categories):
    target_names = dataset.target_names
    selected_idx = [ target_names.index(c) for c in categories  ]
    #필터링
    data_filtered, target_filtered = [], []
    for text,label in zip(dataset.data, dataset.target):
        if label in selected_idx:
            new_label = selected_idx.index(label)  # 라벨 재 정렬
            data_filtered.append(text) ; target_filtered.append( new_label  )
    return data_filtered,target_filtered,categories

In [9]:
# train_data, train_target, target_names = filter_categories(newsgroups_train, categories)
# test_data, test_target, _ = filter_categories(newsgroups_test, categories)


In [10]:
train_data, train_target, target_names = filter_categories(newsgroups_train,categories)
test_data, test_target, _ = filter_categories(newsgroups_test,categories)

In [11]:
import re

def clean_text(text):
    # 헤더 제거
    text = re.sub(r'^From:.*\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'^Subject:.*\n', '', text, flags=re.MULTILINE)

    # 풋터 제거
    text = re.sub(r'\n--\n.*$', '', text, flags=re.DOTALL)

    # 인용문 제거
    text = re.sub(r'(^|\n)[>|:].*', '', text)

    return text

In [12]:
train_data = [ clean_text(t) for t in train_data]
test_data = [ clean_text(t) for t in test_data]

In [13]:
len(train_data), len(train_target), len(test_data), len(test_target)

(2034, 2034, 1353, 1353)

In [14]:
# 멀티 노멀 나이즈베이즈
# multinomal Naive Bayes : 확률 모델
# 문서에 포함된 단어들의 출현 횟수를 기반으로 해서
# 그 문서가 어떤 주제에 속할지 확률을 계산
# 스팸 필터링, 뉴스기사 카테고리, 감성분석
# 베이스 정리 확률 이론 - 조건부 확률

In [15]:
# nltk tokenizrt stemer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Playdata2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# min_df : 단어의 빈도가 최소 5개의 문서에 등장 - 노이즈 감소
# max_df : 50% 너무 흔한 단어는 제거
# 형식:
    # 정수(int)일 때 → “문서 개수 기준”
    # 실수(float, 0~1 사이)일 때 → “문서 비율 기준”

cv = CountVectorizer(max_features=2000, min_df=5, max_df=0.5)

x_train_cv = cv.fit_transform(train_data)
x_test_cv = cv.transform(test_data)
x_train_cv.shape, x_test_cv.shape

((2034, 2000), (1353, 2000))

In [17]:
cv.get_feature_names_out()

array(['00', '000', '01', ..., 'zero', 'zip', 'zoo'],
      shape=(2000,), dtype=object)

In [18]:
(x_train_cv[0].toarray()[0])[x_train_cv[0].toarray()[0]>0]

array([1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 6, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1])

In [19]:
# BOW 기반
# 텍스트 분류의 강력한 baseline 희소데이터에 강함
# 모델 선택
nb = MultinomialNB()
# 학습용데이터 , 벡터데이터
nb.fit(x_train_cv,train_target)
print(nb.score(x_train_cv,train_target), nb.score(x_test_cv,test_target))

# 분류 리포트
from sklearn.metrics import classification_report
y_pred_nb = nb.predict(x_test_cv)
print(classification_report(test_target,y_pred_nb))

0.8549655850540806 0.7642276422764228
              precision    recall  f1-score   support

           0       0.64      0.71      0.67       319
           1       0.66      0.66      0.66       251
           2       0.86      0.88      0.87       389
           3       0.85      0.76      0.80       394

    accuracy                           0.76      1353
   macro avg       0.75      0.75      0.75      1353
weighted avg       0.77      0.76      0.77      1353



In [20]:
# TF-IDF + MNB + LogisticRegression 예제
# TF-IDF: 단어의 중요도를 강조, 흔하게 등장하는 단어의 영향 감소
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# 1️⃣ TF-IDF 벡터화
# max_features=2000 : 상위 2000개의 단어만 사용
# min_df=5 : 최소 5개 문서에 등장한 단어만 사용
# max_df=0.5 : 전체 문서의 50% 이상 등장한 단어는 제외 (너무 흔한 단어)
tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf = tfidf.fit_transform(train_data)  # 학습 데이터로 단어-행렬 생성
x_test_tfidf = tfidf.transform(test_data)        # 테스트 데이터도 동일 벡터화

# 2️⃣ Multinomial Naive Bayes 모델 학습
nb_tfidf = MultinomialNB()
nb_tfidf.fit(x_train_tfidf, train_target)       # 학습 데이터로 모델 학습

# 3️⃣ 정확도 평가
# train score: 학습 데이터에서의 정확도
# test score: 테스트 데이터에서의 정확도
# 만약 train 정확도가 매우 높고 test 정확도가 낮으면 => 과적합(overfitting) 가능성
print(nb_tfidf.score(x_train_tfidf, train_target), nb_tfidf.score(x_test_tfidf, test_target))

0.8967551622418879 0.7634885439763488


In [ ]:
# logistic
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x_train_tfidf, train_target,
                                                test_size=0.2,
                                                stratify=train_target,
                                                random_state=42)
# ^ 여기서 train, test 데이터를 분류한이유, ^

lr = LogisticRegression()
lr.fit(x_train,y_train)
print(lr.score(x_train,y_train),lr.score(x_test,y_test))

# ^ y_train, y_test를 사용하지않은 이유?

0.9496004917025199 0.8304668304668305


In [22]:
# 과적합 해결을 위한 규제 : 규제 알고리즘 RidgeClassifier()
rc = RidgeClassifier(alpha=15)
rc.fit(x_train, y_train)
print(rc.score(x_train,y_train),rc.score(x_test,y_test))

0.8807621389059619 0.7911547911547911


In [23]:
# RidgeClassifier

In [26]:
# L1 규제 L1 Logistic(Lasso^와 유사) 
# 일부 계수를 0으로 만들어서 특성선택을 수행.. 중요피처를 선택하는 효과
l1_lr = LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear')
l1_lr.fit(x_train,y_train)
print(l1_lr.score(x_train,y_train),l1_lr.score(x_test,y_test))

0.8549477566072526 0.773955773955774


c:\Users\Playdata2\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


### 분류 모델 요약

#### 1. RidgeClassifier
- **설명**: 선형 모델에 L2 규제를 적용한 분류기. 과적합을 완화하고 가중치를 안정적으로 학습.
- **특징**:
  - 희소(sparse) 데이터와 잘 맞음
  - 학습 속도가 빠름
  - 규제(alpha)로 과적합 조절 가능

---

#### 2. DecisionTreeClassifier
- **설명**: 데이터를 기준(feature)으로 나누어 트리 구조로 분류하는 모델.
- **특징**:
  - 직관적인 규칙 기반 분류
  - 과적합 위험이 있음 (특히 깊은 트리)
  - 시각화가 쉬움

---

#### 3. RandomForestClassifier
- **설명**: 여러 개의 Decision Tree를 만들어 투표 방식으로 예측하는 앙상블 모델.
- **특징**:
  - 과적합 방지 및 일반화 성능 향상
  - 변수 중요도(feature importance) 제공
  - 안정적이고 강력한 성능

---

#### 4. GradientBoostingClassifier
- **설명**: 이전 트리의 오차를 보정하면서 순차적으로 트리를 학습하는 부스팅 기반 앙상블.
- **특징**:
  - 강력한 예측 성능
  - 학습 속도 느릴 수 있음
  - 과적합 방지를 위해 learning_rate, n_estimators 조절 필요

In [50]:
# 트리모델 + tfidf
tree = DecisionTreeClassifier(criterion='gini',random_state=42,max_features='sqrt')
fores = RandomForestClassifier(criterion='gini',random_state=42,max_depth=3)
gb = GradientBoostingClassifier(n_estimators=200,learning_rate=0.1,max_depth=3)

In [38]:
# 결정트리
tree.fit(x_train_tfidf,train_target)
print(tree.score(x_train_tfidf,train_target),tree.score(x_test_tfidf,test_target))

0.9931170108161259 0.5358462675535847


In [45]:
# 랜덤포레스트
fores.fit(x_train_tfidf,train_target)
print(fores.score(x_train_tfidf,train_target),fores.score(x_test_tfidf,test_target))

0.6465093411996067 0.5846267553584626


In [54]:
# 그라디언트부스팅
# 분할된 train/test 사용
gb.fit(x_train, y_train)

# 학습 데이터 점수
train_score = gb.score(x_train, y_train)

# 테스트 데이터 점수
test_score = gb.score(x_test, y_test)

print(train_score, test_score)


0.9606637984019668 0.7886977886977887


In [63]:
x_train,x_test,y_train,y_test = train_test_split(train_data,train_target,
                 stratify=train_target,test_size=0.2,random_state=42)

In [64]:
# 전처리 
# RegxpTokenizer + stop words + PorterStemmer
english_stops = set(stopwords.words('english'))
regtok = RegexpTokenizer(r"[\w]{3,}")
def custom_tokenizer(text):
    toks = regtok.tokenize(text.lower())
    toks = [t for t in toks if t not in english_stops]
    tokes = [PorterStemmer().stem(t) for t in toks]  # 어간 추출
    return tokes
tfidf_custom = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf_c = tfidf_custom.fit_transform(x_train) # ^ 여기서는 x값만 넣는 이유 ^
x_test_tfidf_c = tfidf_custom.transform(x_test)

c:\Users\Playdata2\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# #

In [74]:
# n-gram 실험
# 성능 향상 기대 연속된 단어 패턴 포착
tfidf_l2 = TfidfVectorizer(token_pattern=r"[\w']{3,}",
                           stop_words = stopwords.words('english'),
                           ngram_range=(1,2),
                           min_df=2, max_df=0.5,
                           # max_features=2000
                           )
x_train_l2 = tfidf_l2.fit_transform(x_train)
x_test_l2 = tfidf_l2.transform(x_test)
# x_train_l2.shape

l1_c = LogisticRegression(max_iter=1000)
l1_c.fit(x_train_l2,y_train)
print(l1_c.score(x_train_l2,y_train),l1_c.score(x_test_l2,y_test))

0.9877074370006146 0.8894348894348895


In [ ]:
# 한국어 처리 konlpy
# 품사기반 태깅 tokenizer : (명사)noun (동사)verb (형용사)adjective
# 데이터 로딩 : 다음 영화 리뷰 데이터셋 불러오기
